In [36]:
import pandas as pd                               # panda
import pandas_datareader as pdr                   # Baixar dados yahoo

import matplotlib.pyplot as plt                   # grafico
import matplotlib.dates as mdates                 # trabalhar datas

import statsmodels.api as sm                      # regressao
from statsmodels.api import add_constant          # para a regressao considerar o intercepto

import math as math
import numpy as np
import scipy.stats


from statsmodels.tsa.stattools import coint       # Funcao Coint ()
from statsmodels.tsa.stattools import adfuller

# Declaracoes
pd.options.display.float_format = '{:.2f}'.format

#!pip install MetaTrader5
#!pip install --upgrade MetaTrader5
import MetaTrader5 as mt5
from datetime import datetime
from datetime import date
from datetime import timedelta
import time
import pytz
import sys

import ipywidgets

%matplotlib inline

In [8]:
def Get_data(tickers,inicio,fim,janela):

    # Checa se ja estamos conectados, se não, conecta
    if (mt5.terminal_info()==None):
          
        # conecte-se ao MetaTrader 5
        if not mt5.initialize():
            print("initialize() failed, error code =",mt5.last_error())
            mt5.shutdown()
        
    #quebra as strings dates(inicio,fim) recebidas e transformando em integer
    inicio_= inicio.split('-')
    inicio = [int(element) for element in inicio_]
    fim_= fim.split('-')
    fim = [int(element) for element in fim_]

    timezone = pytz.timezone("Etc/UTC")
    
    #Verifica caso seja timeframe diario ou intraday
    if janela == '1d':
        dt_inicio = datetime(inicio[0], inicio[1], inicio[2], tzinfo=timezone)
        dt_fim = datetime(fim[0], fim[1], fim[2], tzinfo=timezone)
        timeframe = mt5.TIMEFRAME_D1
    #aqui trata as datas intraday
    else:
        dt_inicio = datetime(inicio[0], inicio[1], inicio[2], inicio[3], inicio[4], tzinfo=timezone)
        dt_fim = datetime(fim[0], fim[1], fim[2], fim[3], fim[4], tzinfo=timezone)
        timeframe = mt5.TIMEFRAME_M15
        
    # obtendo o par de acoes com base nas datas enviadas nas strings, caso len seja 1, getdata retorna o dataframe de uma unica acao
    if (len (tickers) == 1):        
        stock1 = mt5.copy_rates_range(tickers[0], timeframe, dt_inicio, dt_fim)    
        stock_df = pd.DataFrame(stock1)[['close','time']].rename(columns={'close': tickers[0]})
                
    else:    
        stock1 = mt5.copy_rates_range(tickers[0], timeframe, dt_inicio, dt_fim)
        stock2 = mt5.copy_rates_range(tickers[1], timeframe, dt_inicio, dt_fim)
        esquerda = pd.DataFrame(stock1)[['close','time']].rename(columns={'close': tickers[0]})
        direita = pd.DataFrame(stock2)[['close','time']].rename(columns={'close': tickers[1]})
        stock_df = pd.merge(left=esquerda, right=direita, left_on='time', right_on='time')
    
    #Se o timeframe for intraday, retorna um datetime (com minutos e horas), caso contrario, retorna um date
    if (janela != '1d'):
        stock_df['Date'] = stock_df['time'].apply(lambda x: str (datetime.fromtimestamp(x, tz=timezone))[:-9]) #define timezone
    else:
        stock_df['Date'] = stock_df['time'].apply(lambda x: date.fromtimestamp(x+21600)) #adicionando 21600 pra corrigir o fuso no momento de converter tsp em str
        stock_df.drop('time',axis=1,inplace = True)
        
    return stock_df.set_index('Date')

In [9]:
# Gera a REGRESSAO LINEAR MULTIVARIADA (QuantGo "Simples") -> y = b*x + c*t + d + e
def regressao_multivariada(x, y, periodo):  
    
    X = np.column_stack((x, range(1,periodo+1,1)))
    X = sm.add_constant(X)
    res = sm.OLS(y,X).fit()
    coef_ang=res.params[1]
    residuo = y-res.predict()
    zscore  = (residuo - np.mean(residuo))/np.std(residuo);
    return coef_ang, residuo, zscore, res


def regressao_residuos(residuos):
  
    residuos_shifted = residuos.diff(1).fillna(method="bfill")  
    delta = residuos - residuos_shifted
    X = sm.add_constant(residuos_shifted, prepend=True)
    res = sm.OLS(residuos,X).fit()
    return res.params[1]

# ADF da Statsmodel e teste de significancia com os critical values do Ferro
def adftest(df,reg):

    adf  = adfuller(df,maxlag=1, autolag="BIC")
    
    adf_stat = adf[0]
    adf_pvalue = adf[1]
    
    ## %ADF clássico, de acordo com o output da função  - NAO UTILIZADO
    if   adf[0] < adf[4]['1%']:  adfc='99%'
    elif adf[0] < adf[4]['5%']:  adfc='95%'
    elif adf[0] < adf[4]['10%']: adfc='90%'
    else:                        adfc='-'                 

    ## %ADF do Ferro
    # Utiliza duas tabelas - uma para quando o tempo é significante, outra quando não é (indep. do noobs)
    ttest = reg.params[1]/reg.bse[1]
    critical_value_tempo = scipy.stats.t.ppf(0.01,df=(len(df)-3)) # Retorna o inverso bicaudal da distribuição t de Student

    if (abs(ttest) < critical_value_tempo): aceitar_t0 = '99%'
    else:  aceitar_t0 = '0%'
    
    if (aceitar_t0 == '0%'):
        if   adf[0] < -4.32:         adf_sign='99%'
        elif adf[0] < -3.67:         adf_sign='95%'
        elif adf[0] < -3.28:         adf_sign='90%'
        else:                        adf_sign='-' 
    else:    
        if   adf[0] < -3.58:         adf_sign='99%'
        elif adf[0] < -3.22:         adf_sign='95%'
        elif adf[0] < -2.60:         adf_sign='90%'
        else:                        adf_sign='-'                 

    
    
        
    return adf_stat, adf_sign, adf_pvalue  


def pearsonr_ci(x,y,alpha=0.05):
    ''' calculate Pearson correlation along with the confidence interval using scipy and numpy Parameters
    ----------
    x, y : iterable object such as a list or np.array
      Input for correlation calculation
    alpha : float
      Significance level. 0.05 by default
    Returns
    -------
    r : float
      Pearson's correlation coefficient
    pval : float
      The corresponding p value
    lo, hi : float
      The lower and upper bound of confidence intervals
    '''
    
    r, p = scipy.stats.pearsonr(x,y)
    r_z = np.arctanh(r)
    se = 1/np.sqrt(x.size-3)
    z = scipy.stats.norm.ppf(1-alpha/2)
    lo_z, hi_z = r_z-z*se, r_z+z*se
    lo, hi = np.tanh((lo_z, hi_z))
    
    return r, p, lo, hi
    
def pct_financeiro(x,y,coef_ang,residuo):
    
    ultimo_x = float(x.tail(1))
    ultimo_y = float(y.tail(1))
    
    fin_x = ultimo_x*coef_ang
    fin_y = ultimo_y
    
    if (residuo > 0):
        compra = fin_x
        venda = fin_y
    else:
        compra = fin_y
        venda = fin_x
 
    cv = "{:.0%}".format(compra/venda)
    return cv


def calculo_meia_vida1(residuo):
    # Retirado de um paper e adaptado pela formula da planilha do Ferro
    price = pd.Series(residuo)  
    lagged_price = price.shift(1).fillna(method="bfill")  
    delta = price - lagged_price  
    beta = np.polyfit(lagged_price, delta, 1)[0] 
    #half_life = ((-2*np.log(2))/beta)  # paper (varios), mas sem o 2*
    half_life = 2/(-1*np.log(1+beta))   # planilha ferro
     
    return (int(round(half_life)))


In [ ]:
def ajusta_B3Date (data_start, qtde_dias):
    
    df = pd.read_excel('Calendar.xlsx')
    df['Date'] = df['Date'].apply(str)
    try:
        index = df [df['Date'] == data_start].index.values.astype(int)[0]
        data_fim = df.iloc[index+qtde_dias][1]
    except:
        print ('****************************************************************')
        print ('ERRO, TENTATIVA DE ACESSO DE DATAS NAO INCLUIDAS EM CALENDAR.XLS')
        print ('****************************************************************\n')
        sys.exit()
    else:
        return (data_fim + '-17-00')

    
def simula_trade (ticker, beta_inicio, dp_entrada, dp_target, start_time , hlife):
    
    #passando a data de inicio do trade para identificarmos qual a data limite para o trade com base no calendario da B3
    end_time = ajusta_B3Date(start_time[:-6],hlife)
    #print (start_time)
    #print (end_time)
    stock = Get_data(ticker, start_time, end_time,'M5')
    
    return stock
    
def merge_intraday (ticker, dt_inicio, dt_intraday):

    dt_intraday_ant = datetime.strptime(dt_intraday, '%Y-%m-%d-%H-%M') 
    dt_intraday_ant = dt_intraday_ant - timedelta(days=1)  # Converte p/ data e subtrai 1 dia
    dt_fim = dt_intraday_ant.strftime('%Y-%m-%d-%H-%M')
    
    stock  = Get_data(ticker, dt_inicio,dt_fim,'1d')
    stock_intraday = Get_data(ticker, dt_intraday,dt_intraday,'M15')
    
    stock_intraday['Date'] =  stock_intraday['time'].apply(date.fromtimestamp)
    stock_intraday.set_index('Date', inplace = True)
    stock_concat = pd.concat([stock,stock_intraday])
    stock_concat.drop('time',axis=1,inplace = True)

    return stock_concat

In [11]:
def Get_Cubo (tickers, dt_cubo, intra_bt):
    
    #Pegando a data B3 de 250 regoes atras
    if intra_bt:
        dt_inicio = ajusta_B3Date(dt_cubo[:-6],-250)
    else:
        dt_inicio = ajusta_B3Date(dt_cubo,-250)
    
    #print (dt_inicio)
    
    if intra_bt:
        #Para retornar o par com dados diarios, com ultimo dia sendo o intraday passado em dt_final
        par = merge_intraday (tickers, dt_inicio, dt_cubo)
    else:   
        # Busca cotacoes no Yahoo Finance
        par = Get_data(tickers, dt_inicio, dt_cubo,'1d')

    # Cria o cubo de periodos
    cubo = pd.DataFrame(columns=['preco_y','preco_x','ratio','periodo', 'adf_stat','adf_sign','coef_ang',
                                 'desvio','pct_fin','meia_vida', 'corr', 'fx_corr','per_coint','vol_beta_vertical','fx_vol_beta_vertical',
                                 'vol_beta_vertical1','fx_vol_beta_vertical1','adf99_periodos','dp_residuo'])

    # Popula o cubo de periodos
    #for i in range(100,260,10):
    for i in [100,120,140,160,180,200,220,240,250]:

        # Cria subset do tamanho do periodo atual da iteração e seta a variavel indep (x) e dependente (y)
        par_subset = par.tail(i)
        x = par_subset[tickers[1]]
        y = par_subset[tickers[0]]

        # Adiciona o PERIODO analisado no dataframe
        cubo = cubo.append({'periodo': int(i)}, ignore_index=True)
        # Gera a REGRESSAO e retorna o Coef Ang, os residuos e o z-score. 
        # Retorna tbm o vetor de retorno da regressao para ser usado no ADF
        coef_ang, residuo, zscore, reg  = regressao_multivariada(x, y, i)

        # Teste de Estacionariedade dos Resíduos (ADF)
        adf_stat,adf_sign,adf_pvalue = adftest(residuo, reg)
         
        # Calculo do Fisher
        fisher_r, fisher_pvalue, fisher_lo, fisher_hi = pearsonr_ci(x.diff().fillna(method="bfill"), y.diff().fillna(method="bfill"))

        # Calculo do % Financeiro (C/V)
        pct_fin = pct_financeiro(x,y,coef_ang, float(zscore.tail(1)))

        # Calculo da Meia-vida - ORNSTEIN-UHLENBECK
        meia_vida = calculo_meia_vida1(residuo)

        # Correlação 
        corr = np.corrcoef(x.pct_change().cumsum().dropna(), y.pct_change().cumsum().dropna())[1,0]

        # Se fisher minimo menor que 10%, ou se correlacao < 0, faz adf_sign=0%
        if ((fisher_lo <= 0.1) or (corr<=0) or (coef_ang < 0)):
            adf_sign = "0%"
            
        # Preenchimento do cubo
        cubo.loc[ (cubo['periodo']==i), 'preco_y'    ] = float(y.tail(1))
        cubo.loc[ (cubo['periodo']==i), 'preco_x'    ] = float(x.tail(1))
        cubo.loc[ (cubo['periodo']==i), 'ratio'      ] = float(y.tail(1))/float(x.tail(1))
        cubo.loc[ (cubo['periodo']==i), 'coef_ang'   ] = coef_ang 
        cubo.loc[ (cubo['periodo']==i), 'desvio'     ] = float(zscore.tail(1))
        cubo.loc[ (cubo['periodo']==i), 'adf_stat'   ] = adf_stat
        cubo.loc[ (cubo['periodo']==i), 'adf_sign'   ] = adf_sign 
        cubo.loc[ (cubo['periodo']==i), 'pct_fin'    ] = pct_fin
        cubo.loc[ (cubo['periodo']==i), 'meia_vida'  ] = meia_vida
        cubo.loc[ (cubo['periodo']==i), 'corr'       ] = corr
        cubo.loc[ (cubo['periodo']==i), 'fx_corr'    ] = math.ceil(corr*5) # Faixas:  0 a 0.2 / 0.2 a 0.4 / 0.4 a 0.6 / 0.6 a 0.8 / 0.8 a 1.0 ) 
        cubo.loc[ (cubo['periodo']==i), 'dp_residuo' ] = np.std(residuo) 


    ######### Criando algumas variaveis para o backtest
    
    # PER_COINT - Conta quantos periodos estao cointegrados
    cubo_cointegrado = cubo.loc[cubo['adf_sign'] != '-']  
    periodos_coint = len(cubo_cointegrado)                

    # DESVIO_BETA - desvio do coef angular considerando todos os periodos
    desvio_beta = cubo['coef_ang'].std()/cubo['coef_ang'].mean()    
    
    # DESVIO_BETA1 - desvio do coef angular considerando apenas os periodos cointegrados
    desvio_beta1 = cubo_cointegrado['coef_ang'].std()/cubo_cointegrado['coef_ang'].mean() 
    
    # FX_DESVIO_BETA - cria faixas de desvio_beta - até 0.05; entre 0.05 e 0.10; entre 0.10 e 0.20; acima de 0.20
    if   (desvio_beta <= 0.05): fx_desvio_beta = '<= 0.05'
    elif (desvio_beta <= 0.10): fx_desvio_beta = '<= 0.10'
    elif (desvio_beta <= 0.20): fx_desvio_beta = '<= 0.20'
    else: fx_desvio_beta = '> 0.20'
    
    # FX_DESVIO_BETA1 - cria faixas de desvio_beta - até 0.05; entre 0.05 e 0.10; entre 0.10 e 0.20; acima de 0.20
    if   (desvio_beta1 <= 0.05): fx_desvio_beta1 = '<= 0.05'
    elif (desvio_beta1 <= 0.10): fx_desvio_beta1 = '<= 0.10'
    elif (desvio_beta1 <= 0.20): fx_desvio_beta1 = '<= 0.20'
    else: fx_desvio_beta1 = '> 0.20'
    
    # ADFS99 (True/False) - se todos os periodos do cubo tiverem adf=99%, então adf99=1, senao adf99=0
    adf99_periodos = len(cubo.loc[cubo['adf_sign'] == '99%'])  
    
    cubo['per_coint']             = periodos_coint
    cubo['adf99_periodos']        = adf99_periodos
    cubo['vol_beta_vertical']     = desvio_beta
    cubo['fx_vol_beta_vertical']  = fx_desvio_beta
    cubo['vol_beta_vertical1']    = desvio_beta1
    cubo['fx_vol_beta_vertical1'] = fx_desvio_beta1
    
    return cubo

In [12]:
def geraCubodecubos(tickers, dtinicio, dtfim):

    start = time.time()

    #### CUBO DE FECHAMENTOS
    cubo_fechamentos = pd.DataFrame(columns=['data','preco_y','preco_x','ratio','periodo', 'adf_stat','adf_sign','coef_ang',
                                             'desvio','pct_fin','meia_vida', 'corr', 'fx_corr','per_coint','vol_beta_vertical','fx_vol_beta_vertical',
                                             'vol_beta_vertical1','fx_vol_beta_vertical1','adf99_periodos','dp_residuo'])

    datas = Get_data(tickers, dtinicio, dtfim,'1d').reset_index()['Date']

    for dat in datas:
        tmp = Get_Cubo(tickers,str(dat),False)
        tmp['data']=str(dat)+'-19-00'   # Adicionando 19h00 para o cubo de fechamento
        cubo_fechamentos = cubo_fechamentos.append(tmp)


    #### CUBO INTRADAY
    cubo_intraday    = pd.DataFrame(columns=['data','preco_y','preco_x','ratio','periodo', 'adf_stat','adf_sign','coef_ang',
                                             'desvio','pct_fin','meia_vida', 'corr', 'fx_corr','per_coint','vol_beta_vertical','fx_vol_beta_vertical',
                                             'vol_beta_vertical1','fx_vol_beta_vertical1','adf99_periodos','dp_residuo'])

    datahora = Get_data(tickers,str(dtinicio+'-10-00') , str(dtfim+'-17-00'),'15m').reset_index()['Date']

    for dthora in datahora:

        # Convertendo 2018-01-02 10:30 para 2018-01-02-10-30
        dthora1 = dthora.split(' ')
        dthora2 = dthora1[0]+"-"+dthora1[1]
        dthora1 = dthora2.split(':')
        dthora2 = dthora1[0]+"-"+dthora1[1]
        tmp = Get_Cubo(tickers,str(dthora2),True)
        tmp['data']=str(dthora2)
        cubo_intraday = cubo_intraday.append(tmp)


    ### Concatenando e ordenando os dois cubos
    cubo_total = cubo_fechamentos.append(cubo_intraday)
    cubo_total = cubo_total.sort_values(['data','periodo'])
    cubo_total.reset_index(drop=True, inplace=True)

    #display(cubo_total)

    end = time.time()
    print('Tempo total = {total}'.format(total = (end - start)))
    
    return cubo_total

In [93]:
tickers = ['CSNA3','LAME4']

cubototal = geraCubodecubos(tickers, '2020-06-01','2020-07-03')

#cubototal = pd.read_excel('cubo_ITUB4_BBSE3.xlsx')

Tempo total = 275.86730456352234


In [95]:
cubototal.to_excel('cubo_CSNA3_LAME4.xlsx')

In [42]:
def Busca_trade(cubo_of, coint=2, dp=2., adf_sign =0.95, meia_vida = 10, voltando = False):
    
    date_unique = cubo_of['data'].unique()
    a= 0
    
    trade_select    = pd.DataFrame(columns=['data','preco_y','preco_x','ratio','periodo', 'adf_stat','adf_sign','coef_ang',
                                             'desvio','pct_fin','meia_vida', 'corr', 'fx_corr','per_coint','vol_beta_vertical','fx_vol_beta_vertical',
                                             'vol_beta_vertical1','fx_vol_beta_vertical1','adf99_periodos'])

    full_trade_select = pd.DataFrame(columns=['data','preco_y','preco_x','ratio','periodo', 'adf_stat','adf_sign','coef_ang',
                                             'desvio','pct_fin','meia_vida', 'corr', 'fx_corr','per_coint','vol_beta_vertical','fx_vol_beta_vertical',
                                             'vol_beta_vertical1','fx_vol_beta_vertical1','adf99_periodos'])
    
    #arredondando o desvio
    cubo_of['desvio'] = cubo_of['desvio'].apply(lambda x: round(x,2))
    #criando o desvio com valor absoluto
    cubo_of['desvio_abs'] = cubo_of['desvio'].apply(lambda x: abs(x))
    #removendo elementos nao cointegrados adf_sign = '-'
    cubo_of_index = cubo_of[ cubo_of['adf_sign'] == '-' ].index
    cubo_of.drop(cubo_of_index , inplace=True)
    #removendo as linhas de close diario com hora = '19:00'
    cubo_of_index = cubo_of[ cubo_of['data'].apply(lambda x: x[-5:]) == '19-00' ].index
    cubo_of.drop(cubo_of_index , inplace=True)
    #definindo a data quarenta como a primeira data pra iniciar o loop
    dt_quarentena = datetime.strptime(date_unique[0], '%Y-%m-%d-%H-%M')

    for date in date_unique:
        
        cubo_var = cubo_of.loc[cubo_of['data'] == date].copy()
        #selecionando linhas com padroes de DP e #perdiodos cointegrados
        var_trade = cubo_var.loc[(cubo_var['desvio_abs'] >= dp) & (cubo_var['per_coint'] >= coint)]
        # Ordenando por t-stat do ADF e pegando o maior valor
        var_trade = var_trade.sort_values(by='adf_stat')
        
        
        #append do trade de cada iteracao, caso haja
        if len (var_trade) != 0:
            
            #pegando a data do trade e criando o dt_obj da mesma , ESTE SERA O TRADE SELECIONADO
            data_trade = var_trade.iloc[0]['data']
            d_obj_trade = datetime.strptime(data_trade, '%Y-%m-%d-%H-%M') 
            
            #print ('Tentativa de adicionar trade da data    => {data}'.format(data = d_obj_trade)) # remover dps do debug
            #print ('Trades em quarentena ate a data:        => {data}'.format(data = dt_quarentena))# remover dps do debug
            full_trade_select = full_trade_select.append(var_trade.iloc[0]) # remover dps do debug
            
            #verificando se a data do trade e maior que a data de quarentena de 1/4 de meia vida do trade anterior do par
            if (d_obj_trade >= dt_quarentena):
                #print('***************************************')
                #print('********** Trade Adicionado ***********')
                #print('***************************************')
                trade_select = trade_select.append(var_trade.iloc[0])
                #criando o periodo de quarentana arredondado para cima
                dias_quarentena = int(math.ceil(var_trade.iloc[0]['meia_vida'] / 4))
                #criando a date_obj da quarenta a ser reseitada em data util B3
                dt_quarentena = datetime.strptime(ajusta_B3Date(data_trade[:-6], dias_quarentena), '%Y-%m-%d-%H-%M')
            
    #print ('\n')
    
    print ('TOTAL DE TRADES IDENTIFICADOS NO PERIODO  => {data}'.format(data = len(full_trade_select))) # remover dps do debug
    print ('TOTAL DE TRADES SELECIONADOS NO PERIODO   => {data}'.format(data = len(trade_select))) # remover dps do debug
    
    #display(full_trade_select)
    display(trade_select)
    full_trade_select.to_excel('Total_trades.xlsx') # remover dps do debug
    
    return trade_select

In [94]:
trades = Busca_trade (cubototal)

trades.to_excel('Trades_adicionados.xlsx')

TOTAL DE TRADES IDENTIFICADOS NO PERIODO  => 134
TOTAL DE TRADES SELECIONADOS NO PERIODO   => 3


,data,preco_y,preco_x,ratio,periodo,adf_stat,adf_sign,coef_ang,desvio,pct_fin,...,corr,fx_corr,per_coint,vol_beta_vertical,fx_vol_beta_vertical,vol_beta_vertical1,fx_vol_beta_vertical1,adf99_periodos,desvio_abs,dp_residuo
5,2020-06-01-10-00,10.61,28.03,0.38,200.00,-5.67,0%,0.55,2.59,145%,...,-0.01,0.00,9,0.03,<= 0.05,0.03,<= 0.05,3,2.59,0.59
11,2020-06-01-10-15,10.40,28.60,0.36,140.00,-5.01,99%,0.53,2.09,146%,...,0.38,2.00,9,0.03,<= 0.05,0.03,<= 0.05,4,2.09,0.61
20,2020-06-01-10-30,10.38,28.53,0.36,140.00,-5.00,99%,0.53,2.12,146%,...,0.38,2.00,9,0.03,<= 0.05,0.03,<= 0.05,4,2.12,0.61
28,2020-06-01-10-45,10.35,28.94,0.36,120.00,-4.52,99%,0.51,2.04,142%,...,0.55,3.00,9,0.03,<= 0.05,0.03,<= 0.05,5,2.04,0.62
37,2020-06-01-11-00,10.38,28.94,0.36,120.00,-4.51,99%,0.51,2.09,142%,...,0.55,3.00,9,0.03,<= 0.05,0.03,<= 0.05,4,2.09,0.62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2040,2020-06-10-15-45,11.59,30.46,0.38,220.00,-5.71,0%,0.56,2.23,146%,...,-0.05,0.00,9,0.01,<= 0.05,0.01,<= 0.05,4,2.23,0.66
2049,2020-06-10-16-00,11.59,30.38,0.38,220.00,-5.69,0%,0.56,2.30,146%,...,-0.05,0.00,9,0.01,<= 0.05,0.01,<= 0.05,4,2.30,0.66
2058,2020-06-10-16-15,11.66,30.37,0.38,220.00,-5.65,0%,0.56,2.41,145%,...,-0.05,0.00,9,0.01,<= 0.05,0.01,<= 0.05,4,2.41,0.66
2067,2020-06-10-16-30,11.57,30.40,0.38,220.00,-5.70,0%,0.56,2.25,146%,...,-0.05,0.00,9,0.01,<= 0.05,0.01,<= 0.05,4,2.25,0.66


,data,preco_y,preco_x,ratio,periodo,adf_stat,adf_sign,coef_ang,desvio,pct_fin,...,corr,fx_corr,per_coint,vol_beta_vertical,fx_vol_beta_vertical,vol_beta_vertical1,fx_vol_beta_vertical1,adf99_periodos,desvio_abs,dp_residuo
5,2020-06-01-10-00,10.61,28.03,0.38,200.00,-5.67,0%,0.55,2.59,145%,...,-0.01,0.00,9,0.03,<= 0.05,0.03,<= 0.05,3,2.59,0.59
1116,2020-06-05-12-00,11.15,30.63,0.36,100.00,-4.30,95%,0.52,2.06,144%,...,0.67,4.00,9,0.02,<= 0.05,0.02,<= 0.05,5,2.06,0.66
1833,2020-06-10-10-00,12.10,30.44,0.40,220.00,-5.46,0%,0.56,2.98,140%,...,-0.04,0.00,9,0.01,<= 0.05,0.01,<= 0.05,3,2.98,0.66


In [167]:
#removendo as linhas de close diario com hora = '19:00'
cubototal_index = cubototal[ cubototal['data'].apply(lambda x: x[-5:]) == '19-00' ].index
cubototal.drop(cubototal_index , inplace=True)
cubototal['date_obj'] = cubototal['data'].apply(lambda x: datetime.strptime(x,'%Y-%m-%d-%H-%M')) 
#par_periodo = cubototal ['data','preco_y', 'preco_x', 'ratio', 'desvio']
#display (cubototal)

def Simula_Trade (trade_select, cubototal, dp):
    
    #iteracao dentro dos trades selecionados
    for index, trade in trade_select.iterrows():
        
        print ('****************************************************************')
        print ('******************INICIO DE ANALISE DO TRADE********************')
        print ('****************************************************************\n')
        
        #criacao das datas limites para percorrer o trade
        dt_trade = datetime.strptime(trade['data'], '%Y-%m-%d-%H-%M')
        dt_meiavida = datetime.strptime(ajusta_B3Date(trade['data'][:-6], int(trade['meia_vida'])), '%Y-%m-%d-%H-%M')
        periodo_trade = trade['periodo'] #periodo do trade
        #selecionando o periodo do cubo entre as datas limites e o periodo correspondente ao trade selecionado
        cubo_slice = cubototal.loc[(cubototal['date_obj'] > dt_trade) & (cubototal['date_obj'] <= dt_meiavida) & (cubototal['periodo'] == periodo_trade)]
        #criando as variaveis do trade
        cot_x = trade['preco_x']
        cot_y = trade['preco_y']
        ratio_trade = trade['ratio']
        dp_residuo = trade['dp_residuo']*dp
        beta = trade['coef_ang']
        target_fin = dp_residuo
        
        #definindo as ratios conforme o trade tenha sido feito no desvio negativo ou positivo
        if trade_select.loc[index,'desvio'] > 0:
            ratio_gain = (cot_y - dp_residuo)/cot_x
            ratio_loss = (cot_y + dp_residuo)/cot_x
            fin_trade = cot_y - (cot_x*beta)
        else:
            ratio_gain = (cot_y + dp_residuo)/cot_x
            ratio_loss = (cot_y - dp_residuo)/cot_x
            fin_trade = (cot_x*beta) - cot_y
        
        #iterando o trade em questao dentro do periodo da meia-vida
        for var_index , trade_iterado in cubo_slice.iterrows():
            
            cot_x_atual = trade_iterado['preco_x']
            cot_y_atual = trade_iterado['preco_y']
            data_atual = trade_iterado['data']
        
            if trade_select.loc[index,'desvio'] > 0:
                ratio_atual = (cot_y_atual)/cot_x_atual
                fin_atual =  (cot_x_atual*beta) - cot_y_atual
            else:
                ratio_atual = (cot_y_atual)/cot_x_atual
                fin_atual = cot_y_atual - (cot_x_atual*beta)
            
            print('Data do trade => {date}'.format (date=dt_trade))
            print('Data ATUAL do trade => {date}'.format (date=data_atual))
            print('-------------------------')
            print('Ratio inicial => {ratio}'.format(ratio=ratio_trade))            
            print('Ratio atual => {ratio}'.format(ratio=ratio_atual))
            print('Ratio Target Loss => {ratio}'.format(ratio=ratio_loss))
            print('Ratio Target Gain => {ratio}'.format(ratio=ratio_gain))            
            print('-------------------------')
            print('Cotacao inicial X =>{cot}'.format(cot=cot_x))
            print('Cotacao inicial Y =>{cot}'.format(cot=cot_y))
            print('Cotacao atual X =>{cot}'.format(cot=cot_x_atual))
            print('Cotacao atual Y =>{cot}'.format(cot=cot_y_atual))
            print('-------------------------')            
            print('Financeiro inicial => {fin}'.format(fin=fin_trade))
            print('Financeiro atual => {fin}'.format(fin=fin_atual))
            print('Financeiro acumulado => {fin}'.format(fin=fin_atual+fin_trade))
            print('Target Financeiro => {fin}'.format(fin=target_fin))
            print('\n')
            print('\n')
            
            #trade_select.loc[index,'result_fin']
            
            #trade_select.loc[index,'result_ratio'] = index 
    return

In [168]:
Simula_Trade(trades, cubototal, 1)

****************************************************************
******************INICIO DE ANALISE DO TRADE********************
****************************************************************

Data do trade => 2020-06-01 10:00:00
Data ATUAL do trade => 2020-06-01-10-15
-------------------------
Ratio inicial => 0.378523011059579
Ratio atual => 0.36363636363636365
Ratio Target Loss => 0.39970773996231745
Ratio Target Gain => 0.3573382821568405
-------------------------
Cotacao inicial X =>28.03
Cotacao inicial Y =>10.61
Cotacao atual X =>28.6
Cotacao atual Y =>10.4
-------------------------
Financeiro inicial => -4.7770519874174155
Financeiro atual => 5.299953151628186
Financeiro acumulado => 0.5229011642107704
Target Financeiro => 0.593807951143759




Data do trade => 2020-06-01 10:00:00
Data ATUAL do trade => 2020-06-01-10-30
-------------------------
Ratio inicial => 0.378523011059579
Ratio atual => 0.3638275499474238
Ratio Target Loss => 0.39970773996231745
Ratio Target Gain => 

-------------------------
Ratio inicial => 0.378523011059579
Ratio atual => 0.353175922846691
Ratio Target Loss => 0.39970773996231745
Ratio Target Gain => 0.3573382821568405
-------------------------
Cotacao inicial X =>28.03
Cotacao inicial Y =>10.61
Cotacao atual X =>30.07
Cotacao atual Y =>10.62
-------------------------
Financeiro inicial => -4.7770519874174155
Financeiro atual => 5.88690878564544
Financeiro acumulado => 1.1098567982280247
Target Financeiro => 0.593807951143759




Data do trade => 2020-06-01 10:00:00
Data ATUAL do trade => 2020-06-02-16-15
-------------------------
Ratio inicial => 0.378523011059579
Ratio atual => 0.3562374916611074
Ratio Target Loss => 0.39970773996231745
Ratio Target Gain => 0.3573382821568405
-------------------------
Cotacao inicial X =>28.03
Cotacao inicial Y =>10.61
Cotacao atual X =>29.98
Cotacao atual Y =>10.68
-------------------------
Financeiro inicial => -4.7770519874174155
Financeiro atual => 5.77750333866479
Financeiro acumulado => 

Cotacao inicial X =>28.03
Cotacao inicial Y =>10.61
Cotacao atual X =>29.38
Cotacao atual Y =>11.17
-------------------------
Financeiro inicial => -4.7770519874174155
Financeiro atual => 4.958133692127136
Financeiro acumulado => 0.18108170470972063
Target Financeiro => 0.593807951143759




Data do trade => 2020-06-01 10:00:00
Data ATUAL do trade => 2020-06-05-14-00
-------------------------
Ratio inicial => 0.378523011059579
Ratio atual => 0.3820803295571576
Ratio Target Loss => 0.39970773996231745
Ratio Target Gain => 0.3573382821568405
-------------------------
Cotacao inicial X =>28.03
Cotacao inicial Y =>10.61
Cotacao atual X =>29.13
Cotacao atual Y =>11.13
-------------------------
Financeiro inicial => -4.7770519874174155
Financeiro atual => 4.860896339403112
Financeiro acumulado => 0.08384435198569662
Target Financeiro => 0.593807951143759




Data do trade => 2020-06-01 10:00:00
Data ATUAL do trade => 2020-06-05-14-15
-------------------------
Ratio inicial => 0.3785230110595

Data do trade => 2020-06-05 12:00:00
Data ATUAL do trade => 2020-06-05-12-15
-------------------------
Ratio inicial => 0.36402220045706823
Ratio atual => 0.36419954352787737
Ratio Target Loss => 0.38561729514634985
Ratio Target Gain => 0.3424271057677866
-------------------------
Cotacao inicial X =>30.63
Cotacao inicial Y =>11.15
Cotacao atual X =>30.67
Cotacao atual Y =>11.17
-------------------------
Financeiro inicial => -4.863881266159774
Financeiro atual => 4.864793941662432
Financeiro acumulado => 0.0009126755026578337
Target Financeiro => 0.6614577503326962




Data do trade => 2020-06-05 12:00:00
Data ATUAL do trade => 2020-06-05-12-30
-------------------------
Ratio inicial => 0.36402220045706823
Ratio atual => 0.3642297650130548
Ratio Target Loss => 0.38561729514634985
Ratio Target Gain => 0.3424271057677866
-------------------------
Cotacao inicial X =>30.63
Cotacao inicial Y =>11.15
Cotacao atual X =>30.64
Cotacao atual Y =>11.16
-------------------------
Financeiro inici

Data do trade => 2020-06-05 12:00:00
Data ATUAL do trade => 2020-06-10-11-00
-------------------------
Ratio inicial => 0.36402220045706823
Ratio atual => 0.3833824010467779
Ratio Target Loss => 0.38561729514634985
Ratio Target Gain => 0.3424271057677866
-------------------------
Cotacao inicial X =>30.63
Cotacao inicial Y =>11.15
Cotacao atual X =>30.57
Cotacao atual Y =>11.72
-------------------------
Financeiro inicial => -4.863881266159774
Financeiro atual => 4.262512252905788
Financeiro acumulado => -0.6013690132539864
Target Financeiro => 0.6614577503326962




Data do trade => 2020-06-05 12:00:00
Data ATUAL do trade => 2020-06-10-11-15
-------------------------
Ratio inicial => 0.36402220045706823
Ratio atual => 0.3858839050131926
Ratio Target Loss => 0.38561729514634985
Ratio Target Gain => 0.3424271057677866
-------------------------
Cotacao inicial X =>30.63
Cotacao inicial Y =>11.15
Cotacao atual X =>30.32
Cotacao atual Y =>11.7
-------------------------
Financeiro inicial =

Data do trade => 2020-06-10 10:00:00
Data ATUAL do trade => 2020-06-10-10-15
-------------------------
Ratio inicial => 0.39750328515111694
Ratio atual => 0.39532587228439764
Ratio Target Loss => 0.41924218782963535
Ratio Target Gain => 0.37576438247259847
-------------------------
Cotacao inicial X =>30.44
Cotacao inicial Y =>12.1
Cotacao atual X =>30.38
Cotacao atual Y =>12.01
-------------------------
Financeiro inicial => -4.883293751639046
Financeiro atual => 4.939818139776419
Financeiro acumulado => 0.05652438813737248
Target Financeiro => 0.6617321975341023




Data do trade => 2020-06-10 10:00:00
Data ATUAL do trade => 2020-06-10-10-30
-------------------------
Ratio inicial => 0.39750328515111694
Ratio atual => 0.3928806855636124
Ratio Target Loss => 0.41924218782963535
Ratio Target Gain => 0.37576438247259847
-------------------------
Cotacao inicial X =>30.44
Cotacao inicial Y =>12.1
Cotacao atual X =>30.34
Cotacao atual Y =>11.92
-------------------------
Financeiro inicial

Cotacao atual X =>31.6
Cotacao atual Y =>10.7
-------------------------
Financeiro inicial => -4.883293751639046
Financeiro atual => 6.930488914316488
Financeiro acumulado => 2.0471951626774416
Target Financeiro => 0.6617321975341023




Data do trade => 2020-06-10 10:00:00
Data ATUAL do trade => 2020-06-15-16-15
-------------------------
Ratio inicial => 0.39750328515111694
Ratio atual => 0.3388272583201268
Ratio Target Loss => 0.41924218782963535
Ratio Target Gain => 0.37576438247259847
-------------------------
Cotacao inicial X =>30.44
Cotacao inicial Y =>12.1
Cotacao atual X =>31.55
Cotacao atual Y =>10.69
-------------------------
Financeiro inicial => -4.883293751639046
Financeiro atual => 6.912592571097635
Financeiro acumulado => 2.029298819458589
Target Financeiro => 0.6617321975341023




Data do trade => 2020-06-10 10:00:00
Data ATUAL do trade => 2020-06-15-16-30
-------------------------
Ratio inicial => 0.39750328515111694
Ratio atual => 0.33396345305608066
Ratio Target Lo



Data do trade => 2020-06-10 10:00:00
Data ATUAL do trade => 2020-06-18-14-30
-------------------------
Ratio inicial => 0.39750328515111694
Ratio atual => 0.3427635497319833
Ratio Target Loss => 0.41924218782963535
Ratio Target Gain => 0.37576438247259847
-------------------------
Cotacao inicial X =>30.44
Cotacao inicial Y =>12.1
Cotacao atual X =>33.58
Cotacao atual Y =>11.51
-------------------------
Financeiro inicial => -4.883293751639046
Financeiro atual => 7.225184105783152
Financeiro acumulado => 2.3418903541441054
Target Financeiro => 0.6617321975341023




Data do trade => 2020-06-10 10:00:00
Data ATUAL do trade => 2020-06-18-14-45
-------------------------
Ratio inicial => 0.39750328515111694
Ratio atual => 0.341870160810006
Ratio Target Loss => 0.41924218782963535
Ratio Target Gain => 0.37576438247259847
-------------------------
Cotacao inicial X =>30.44
Cotacao inicial Y =>12.1
Cotacao atual X =>33.58
Cotacao atual Y =>11.48
-------------------------
Financeiro inicial 

In [29]:
dt_quarentena = datetime.strptime('2020-08-11', '%Y-%m-%d')

datetime.now()

datetime.datetime(2020, 8, 17, 10, 39, 43, 601378)

In [30]:
dt_quarentena

datetime.datetime(2020, 8, 11, 0, 0)

In [31]:
if dt_quarentena > datetime.now():
    print ('Erro')